### Coleta de Dados


In [ ]:
#Bibliotecas e Variaveis Uteis
import geopandas as gpd
from calendar import monthrange
import pandas as pd
import numpy as np
import xarray as xr
import backoff
import cdsapi
import os
import time
import zipfile
import shutil
from glob import glob
from tqdm import tqdm
import netCDF4
import h5netcdf
import requests
import duckdb
from pathlib import Path
from datetime import datetime, timedelta
import subprocess
import sys
import logging


#Caminhos:
pasta_dados = os.path.join('Dados')

pasta_era5 = os.path.join(pasta_dados, 'ERA5_Bruto')


serie_temporal = os.path.join(pasta_dados, 'Series_Temporais')
tratado_era5 = os.path.join(serie_temporal,  'series_temporais_bruto.parquet')
dataset_locais = os.path.join(pasta_dados, 'Dataset_Locais.csv')


#Variaveis Uteis

subsistema_nordeste_ufs = [
    'MA',  # Maranhão
    'PI',  # Piauí
    'CE',  # Ceará
    'RN',  # Rio Grande do Norte
    'PB',  # Paraíba
    'PE',  # Pernambuco
    'AL',  # Alagoas
    'SE',  # Sergipe
    'BA'   # Bahia
]

status_operacional = [
    'Operação', 'Construção não iniciada', 'Construção', 'DRO'
    ]
#DRO -> Despacho de Requerimento de Outorga -> Entrara no longo prazo (talvez mais de 5 anos)
#Construção não iniciada -> Entrara no médio prazo (menos de 5 anos)
#Construção -> Entrara no curto prazo (de 1-3 anos)


##### Localização dos Parques
* Potencia em Kw
* Enriquecimento de dados para encontrar a altura de cada turbina (coord)

In [10]:
#Datasets menores não usados

caminho_shapefile_Existe = 'Dados\LocParquesEol\EOL_-_Base_Existente.shp'
caminho_shapefile_Planejada = 'Dados\LocParquesEol\EOL_-_Expansão_Planejada.shp'
caminho_shapefile_Anemometro = 'Dados\LocParquesEol\Torres_Anemométricas_do_AMA.shp'

lst_files = [caminho_shapefile_Existe, caminho_shapefile_Planejada, caminho_shapefile_Anemometro]
lst_case = ["Existe", "Planejada", "Anemometro"]
i = -1

for caminho_shapefile in lst_files:

    try:

        gdf_parques = gpd.read_file(caminho_shapefile)
        print(f"passando aqui{i}")
        print(gdf_parques.columns)
        print(gdf_parques.head(2))




        gdf_parques['Longitude'] = gdf_parques.geometry.x
        gdf_parques['Latitude'] = gdf_parques.geometry.y





        nome_coluna_usina = 'Nome'
        if nome_coluna_usina not in gdf_parques.columns:

            nome_coluna_usina = gdf_parques.columns[0]
            print("Coluna 'Nome' não encontrada. Usando a primeira coluna como referência de nome.")
        
        i+=1

        if i == 0:
            gdf_parques['ano_init'] = pd.to_datetime(gdf_parques['ini_oper'], errors='coerce').dt.year
            #CONVERSÃO DE KW P MW
            gdf_parques['potencia'] = gdf_parques['potencia'] / 1000
            
            ##Formatação CEG
            gdf_parques['ceg'] = (
                        gdf_parques['ceg']
                        .str.rsplit('.', n=1)         
                        .str.join('-')                
)
            gdf_parques['ceg'] = gdf_parques['ceg'].str.replace('.', "",)
            coordenadas_df = gdf_parques[[nome_coluna_usina, 'potencia', 'ano_init', 'Latitude', 'Longitude', 'ceg']]
        
        elif i == 1:
            gdf_parques['ano_previsto'] = pd.to_datetime(gdf_parques['ano_prev'], errors='coerce').dt.year
            nome_coluna_usina = gdf_parques.columns[1]
            #CONVERSÃO DE KW P MW
            gdf_parques['potencia'] = gdf_parques['potencia'] / 1000
            
            ##Formatação CEG
            gdf_parques['ceg'] = (
                        gdf_parques['ceg']
                        .str.rsplit('.', n=1)         
                        .str.join('-')                
)
            gdf_parques['ceg'] = gdf_parques['ceg'].str.replace('.', "",)
            coordenadas_df = gdf_parques[[nome_coluna_usina, 'potencia', 'ano_previsto', 'Latitude', 'Longitude', 'ceg']]
            
        else:
            coordenadas_df = gdf_parques[[nome_coluna_usina, 'Latitude', 'Longitude',]]


        print("Coordenadas Extraídas com Sucesso!\n")

        print(f"\nTotal de {len(coordenadas_df)} parques eólicos encontrados.")

        print("Salvando em csv...")
        name = lst_case[lst_files.index(caminho_shapefile)]

        coordenadas_df.to_csv('Dados/LocParquesEol/' + name + '.csv', index=False, encoding='utf-8-sig')



    except Exception as e:
        print(f"Ocorreu um erro: {e}")

passando aqui-1
Index(['Nome', 'potencia', 'propriet', 'ini_oper', 'ceg', 'created_us',
       'created_da', 'last_edite', 'last_edi_1', 'geometry'],
      dtype='object')
                  Nome  potencia  \
0  RN 15 - Rio do Fogo   49300.0   
1           Millennium   10200.0   

                                            propriet    ini_oper  \
0  100% para ENERGIAS RENOVÁVEIS DO BRASIL S/A (PIE)  2006/07/15   
1  100% para MILLENNIUM CENTRAL GERADORA EÓLICA S...  2007/11/28   

                     ceg         created_us created_da         last_edite  \
0  EOL.CV.RN.028424-6.01  BERNARDO.OLIVEIRA 2024-12-13  BERNARDO.OLIVEIRA   
1  EOL.CV.PB.029041-6.01  BERNARDO.OLIVEIRA 2024-12-13  BERNARDO.OLIVEIRA   

  last_edi_1                    geometry  
0 2024-12-13   POINT (-35.38671 -5.3097)  
1 2024-12-13  POINT (-34.97194 -6.49389)  
Coordenadas Extraídas com Sucesso!


Total de 981 parques eólicos encontrados.
Salvando em csv...
passando aqui0
Index(['leilao', 'nome', 'ceg', 'ano_pre

In [ ]:
#Criação dos Datasets grandes

# --- Definição dos Caminhos ---

arquivos_para_processar = {
    "CentraisGeradoras": os.path.join(pasta_dados, 'Central_Geradora_Eólica_-_EOL.shp'),
    "Aerogeradores": os.path.join(pasta_dados, 'Aerogeradores.shp'),
    "AerogeradoresAlteracao": os.path.join(pasta_dados, 'Aerogeradores_com_solicitação_de_alteração_de_projetos.shp'),
    "PoligonosParques": os.path.join(pasta_dados, 'Polígono_do_Parque_Eólico_-_EOL.shp')
}

print("Iniciando extração de dados...")

# --- Loop de Processamento ---
for nome_saida, caminho_shapefile in arquivos_para_processar.items():
    
    print(f"\n--- Processando arquivo para: {nome_saida} ---")
    
    try:
        gdf = gpd.read_file(caminho_shapefile, encoding='utf-8')
        print(gdf.columns)
        
        df_final = None

        if nome_saida == "CentraisGeradoras":
            colunas_principais = ['CEG', 'Nome', 'P_FISC_KW', 'FASE', 'UF1', 'QTD_AEG', 'INICIO_OPE', 'geometry']
            df_final = gdf[colunas_principais].copy()
            
            # Adicionando e renomeando colunas para padronização
            df_final['potencia_mw'] = df_final['P_FISC_KW'] / 1000
            df_final.rename(columns={'Nome': 'nome_eol', 'FASE': 'status_operacao', 'UF1': 'uf', 'QTD_AEG':'qtd_aerogeradores'}, inplace=True)
            
            # Converte a data de início de operação para extrair apenas o ano
            df_final['ano_inicio_operacao'] = pd.to_datetime(df_final['INICIO_OPE'], errors='coerce').dt.year

            df_final['longitude'] = df_final.geometry.x
            df_final['latitude'] = df_final.geometry.y
            df_final.drop(columns=['geometry', 'P_FISC_KW', 'INICIO_OPE'], inplace=True)
            
            # Filtros
            df_final = df_final[df_final['uf'].isin(subsistema_nordeste_ufs)]
            df_final = df_final[df_final['status_operacao'].isin(status_operacional)]

        elif nome_saida == "Aerogeradores" or nome_saida == "AerogeradoresAlteracao":
            colunas_principais = ['CEG', 'ALT_TORRE', 'DIAM_ROTOR', 'POT_MW', 'NOME_EOL', 'DEN_AEG', 'OPERACAO', 'UF', 'geometry']
            df_final = gdf[colunas_principais].copy()
            
            df_final['longitude'] = df_final.geometry.x
            df_final['latitude'] = df_final.geometry.y
            
            df_final.rename(columns={
                'POT_MW': 'potencia_turbina_mw', 
                'ALT_TORRE':'altura_torre_m', 
                'DIAM_ROTOR':'diametro_rotor_m',
                'NOME_EOL': 'nome_eol',
                'DEN_AEG': 'denominacao_aerogerador',
                'OPERACAO': 'status_operacao_turbina',
                'UF': 'uf'
            }, inplace=True)
            df_final.drop(columns=['geometry'], inplace=True)

        elif nome_saida == "PoligonosParques":
            # Removida a coluna 'CEG' que não existe neste arquivo
            colunas_principais = ['NOME_EOL', 'POT_MW', 'AREA_M2', 'geometry']
            df_final = gdf[colunas_principais].copy()
            
            df_final['centroid_lon'] = df_final.geometry.centroid.x
            df_final['centroid_lat'] = df_final.geometry.centroid.y
            df_final.rename(columns={'NOME_EOL': 'nome_eol', 'POT_MW': 'potencia_mw', 'AREA_M2':'area_m2'}, inplace=True)


        if df_final is not None:
            caminho_saida = os.path.join(pasta_dados, f'{nome_saida}.csv')
            df_final.to_csv(caminho_saida, index=False, encoding='utf-8-sig')
            print(f"Dados extraídos com sucesso! {len(df_final)} registros salvos em '{caminho_saida}'")

    except Exception as e:
        print(f"!!!!!! Ocorreu um erro ao processar o arquivo para '{nome_saida}': {e}")
        continue

print("\n--- Extração de todos os arquivos concluída! ---")

Iniciando extração de dados...

--- Processando arquivo para: CentraisGeradoras ---


Index(['Nome', 'CODMUN', 'MUNIC1', 'UF1', 'PROC_ANEEL', 'ATO_LEGAL',
       'P_OUT_KW', 'P_FISC_KW', 'PROPRIETAR', 'CaminhoFot', 'CaminhoCon',
       'DATA_ATUAL', 'EOL_VERSAO', 'DRO_ID_SCG', 'LONG_GMS', 'LAT_GMS',
       'NOMES_EOL_', 'DRO_DATA', 'ID_LT', 'ID_SE', 'ID_EST', 'DRO_DT_VIG',
       'CEG', 'ID_EMPREEN', 'LAT_DEC', 'LONG_DEC', 'QTD_AEG', 'INICIO_OPE',
       'FASE', 'VALIDACAO', 'FUSO', 'DATUM_EMP', 'TIPO_EXP', 'OBS',
       'geometry'],
      dtype='object')
Dados extraídos com sucesso! 2423 registros salvos em 'Dados\LocParquesEol1\CentraisGeradoras.csv'

--- Processando arquivo para: Aerogeradores ---
Index(['POT_MW', 'ALT_TOTAL', 'ALT_TORRE', 'DIAM_ROTOR', 'DATA_ATUAL',
       'EOL_VERSAO', 'NOME_EOL', 'DEN_AEG', 'X', 'Y', 'DATUM_EMP', 'OPERACAO',
       'UF', 'FUSO_AG', 'PROPRIETAR', 'CEG', 'ORIGEM', 'geometry'],
      dtype='object')
Dados extraídos com sucesso! 20209 registros salvos em 'Dados\LocParquesEol1\Aerogeradores.csv'

--- Processando arquivo para: Aerogerad

C:\Users\Gustavo\AppData\Local\Temp\ipykernel_25416\451817473.py:68: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df_final['centroid_lon'] = df_final.geometry.centroid.x
C:\Users\Gustavo\AppData\Local\Temp\ipykernel_25416\451817473.py:69: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df_final['centroid_lat'] = df_final.geometry.centroid.y


Dados extraídos com sucesso! 2245 registros salvos em 'Dados\LocParquesEol1\PoligonosParques.csv'

--- Extração de todos os arquivos concluída! ---


In [ ]:
#Junção dos datasets de ParquesEolicos com Aerogeradoes ~~ 20K linhas




caminho_centrais = os.path.join(pasta_dados, 'CentraisGeradoras.csv')
caminho_aerogeradores = os.path.join(pasta_dados, 'Aerogeradores.csv')
pasta_destino = os.path.join('Dados')

try:
    # 1. Carregar os dois datasets principais
    df_parques = pd.read_csv(caminho_centrais)
    df_turbinas = pd.read_csv(caminho_aerogeradores)
    print(f"Carregados {len(df_parques)} parques e {len(df_turbinas)} turbinas.")

    # 2. Merge
    # "Para cada parque na minha lista da esquerda (df_parques),
    # adicione as informações das turbinas da direita (df_turbinas)
    # que tiverem o mesmo 'CEG'."
    df_mestre = pd.merge(
        left=df_parques,
        right=df_turbinas,
        on='CEG',
        how='left'
    )

    # 3. Analisar o resultado
    print("\n--- DATASET MESTRE CRIADO! ---")
    print(f"O dataset final tem {len(df_mestre)} linhas.")
    
    # Renomeando colunas para evitar conflito de nomes (ex: nome_eol_x, nome_eol_y)
    # e selecionando a ordem final das colunas para maior clareza.
    print(df_mestre.columns.tolist())
    df_mestre = df_mestre.rename(columns={'nome_eol_x': 'nome_parque', 'latitude_x': 'latitude_parque', 'longitude_x': 'longitude_parque',
                              'latitude_y': 'latitude_turbina', 'longitude_y': 'longitude_turbina', 'uf_x': 'uf', 'potencia_mw': 'potencia_parque_mw'}
                              )
    print(df_mestre.columns.tolist())
    
    df_mestre = df_mestre.drop(columns=['uf_y', 'nome_eol_y'],)

    
    # Verificando como ficaram os parques futuros (sem turbinas)
    parques_futuros = df_mestre[df_mestre['altura_torre_m'].isna()]
    print(f"\nEncontrados {len(parques_futuros)} registros de parques futuros (sem dados de turbinas).")
    print("Exemplo de um parque futuro no dataset mestre:")
    print(parques_futuros[['nome_parque', 'status_operacao', 'potencia_parque_mw', 'altura_torre_m']].head(2).to_string())

    # Verificando como ficaram os parques existentes
    print("\nExemplo de um parque existente no dataset mestre (dados do parque repetidos para cada turbina):")
    print(df_mestre[['nome_parque', 'status_operacao', 'potencia_turbina_mw', 'altura_torre_m']].head(2).to_string())
    
    # 4. Salvar o dataset mestre
    caminho_saida = os.path.join(pasta_destino, 'Dataset_Locais.csv')
    df_mestre.to_csv(caminho_saida, index=False, encoding='utf-8-sig')
    print(f"\nDataset mestre salvo em '{caminho_saida}'")


except Exception as e:
    print(f"!!!!!! Ocorreu um erro: {e}")

Carregados 2423 parques e 20209 turbinas.

--- DATASET MESTRE CRIADO! ---
O dataset final tem 17758 linhas.
['CEG', 'nome_eol_x', 'status_operacao', 'uf_x', 'qtd_aerogeradores', 'potencia_mw', 'ano_inicio_operacao', 'longitude_x', 'latitude_x', 'altura_torre_m', 'diametro_rotor_m', 'potencia_turbina_mw', 'nome_eol_y', 'denominacao_aerogerador', 'status_operacao_turbina', 'uf_y', 'longitude_y', 'latitude_y']
['CEG', 'nome_parque', 'status_operacao', 'uf', 'qtd_aerogeradores', 'potencia_parque_mw', 'ano_inicio_operacao', 'longitude_parque', 'latitude_parque', 'altura_torre_m', 'diametro_rotor_m', 'potencia_turbina_mw', 'nome_eol_y', 'denominacao_aerogerador', 'status_operacao_turbina', 'uf_y', 'longitude_turbina', 'latitude_turbina']

Encontrados 587 registros de parques futuros (sem dados de turbinas).
Exemplo de um parque futuro no dataset mestre:
                                                             nome_parque status_operacao  potencia_parque_mw  altura_torre_m
6069  Sistema H

In [ ]:
# Coleta ERA5
#pasta_dados = os.path.join('Dados', 'LocParquesEol1')

try:
    import netCDF4
    print("netCDF4 disponível")
except ImportError:
    try:
        import h5netcdf
        print("h5netcdf disponível")
    except ImportError:
        print("AVISO: Nem netCDF4 nem h5netcdf estão instalados. Instale um deles:")
        print("pip install netcdf4")
        print("ou")
        print("pip install h5netcdf")



# Local de Salvamento

os.makedirs(pasta_era5, exist_ok=True)

# Calculando Area de Interesse
lat_max = dataset_locais['latitude_turbina'].max() + 0.5
lat_min = dataset_locais['latitude_turbina'].min() - 0.5
lon_max = dataset_locais['longitude_turbina'].max() + 0.5
lon_min = dataset_locais['longitude_turbina'].min() - 0.5

boundBox = [lat_max, lon_min, lat_min, lon_max]  # N/W/S/E
print(f"Area de Download Calculada: {boundBox}")

# Request
c = cdsapi.Client()

vars = [
    "10m_u_component_of_wind",
    "10m_v_component_of_wind",
     "100m_u_component_of_wind",
     "100m_v_component_of_wind",
    "2m_temperature",
    "sea_surface_temperature",
    "surface_pressure",
    "total_precipitation",
    "friction_velocity"
]

# Configuração de download
anos = range(2000, 2026); meses = [str(m).zfill(2) for m in range(1, 13)]
#anos = [2024]; meses = ['06']

lst_errors = []

print("Download Iniciado...")

# Função de download com backoff corrigida
@backoff.on_exception(backoff.expo,
                      Exception,
                      max_tries=3,
                      base=2,
                      max_time=4000)
def retrieve_and_validate(ano: int, mes: str, output_file: str):
    # Calcular dias válidos para o mês/ano
    _, last_day = monthrange(ano, int(mes))
    dias_validos = [str(d).zfill(2) for d in range(1, last_day + 1)]
    
    c.retrieve('reanalysis-era5-single-levels', {
        'product_type': 'reanalysis',
        'variable': vars,
        'area': boundBox,
        'year': str(ano),
        'month': mes,
        'day': dias_validos,
        'time': [f"{h:02d}:00" for h in range(24)],
        'data_format': 'netcdf'  # Usar exatamente como mostrado no site do CDS
    }, output_file)

def process_downloaded_file(file_path):
    """
    Processa arquivo baixado, extraindo ZIP se necessário e fazendo merge
    """
    try:
        # Verifica se é um arquivo ZIP
        if zipfile.is_zipfile(file_path):
            print(f"Arquivo ZIP detectado: {file_path}")
            
            # Cria diretório temporário para extração
            extract_dir = file_path.replace('.nc', '_temp_extract')
            os.makedirs(extract_dir, exist_ok=True)
            
            # Extrai o ZIP
            with zipfile.ZipFile(file_path, 'r') as zip_file:
                zip_contents = zip_file.namelist()
                print(f"Conteúdo do ZIP: {zip_contents}")
                zip_file.extractall(extract_dir)
            
            # Encontra arquivos .nc extraídos
            nc_files = [f for f in zip_contents if f.endswith('.nc')]
            
            if len(nc_files) == 0:
                raise Exception("Nenhum arquivo NetCDF encontrado no ZIP")
            elif len(nc_files) == 1:
                # Apenas um arquivo - move para o local final
                src_file = os.path.join(extract_dir, nc_files[0])
                # Remove o ZIP original
                os.remove(file_path)
                # Move o arquivo extraído para o local final
                shutil.move(src_file, file_path)
                print(f"Arquivo único extraído: {nc_files[0]}")
            else:
                # Múltiplos arquivos - faz merge
                print(f"Fazendo merge de {len(nc_files)} arquivos NetCDF...")
                
                datasets = []
                for nc_file in nc_files:
                    nc_path = os.path.join(extract_dir, nc_file)
                    print(f"Carregando: {nc_file}")
                    ds = xr.open_dataset(nc_path)
                    datasets.append(ds)
                
                # Faz o merge dos datasets
                print("Combinando datasets...")
                try:
                    # Tenta merge primeiro
                    combined = xr.merge(datasets)
                except Exception as e:
                    print(f"Merge falhou, tentando concatenar: {e}")
                    # Se merge falhar, tenta concatenar por time
                    combined = xr.concat(datasets, dim='time')
                
                # Remove arquivo ZIP original
                os.remove(file_path)
                
                # Salva o arquivo combinado
                print(f"Salvando arquivo combinado: {file_path}")
                combined.to_netcdf(file_path)
                
                # Fecha todos os datasets
                for ds in datasets:
                    ds.close()
                combined.close()
                
                print(f"Merge concluído com sucesso")
            
            # Limpa diretório temporário
            shutil.rmtree(extract_dir, ignore_errors=True)
            return True
            
        else:
            # Não é ZIP, arquivo já está no formato correto
            print(f"Arquivo NetCDF direto (não é ZIP): {file_path}")
            return True
            
    except Exception as e:
        print(f"Erro ao processar arquivo {file_path}: {e}")
        # Limpa arquivos temporários em caso de erro
        if 'extract_dir' in locals() and os.path.exists(extract_dir):
            shutil.rmtree(extract_dir, ignore_errors=True)
        return False


def validate_netcdf(file_path):
    """Valida se o arquivo NetCDF foi baixado corretamente"""
    try:
        with xr.open_dataset(file_path) as ds:
            # Verifica se contém pelo menos algumas variáveis esperadas
            expected_vars = ['u10', 'v10', 't2m', 'sp', 'tp']  # Variáveis mais comuns
            found_vars = [var for var in expected_vars if var in ds.variables]
            
            if len(found_vars) == 0:
                print(f"Nenhuma variável esperada encontrada no arquivo")
                print(f"Variáveis disponíveis: {list(ds.data_vars)}")
                return False
            
            print(f"Arquivo válido. Dimensões: {dict(ds.sizes)}")
            print(f"Variáveis encontradas: {found_vars}")
            return True
    except Exception as e:
        print(f"Erro ao validar arquivo {file_path}: {e}")
        return False

for ano in anos:
    for mes in meses:
        # Mudança: salvar como .nc ao invés de .zip
        nome_arq = f"ERA5_NE-Brasil_{ano}-{mes}.nc"
        output_file = os.path.join(pasta_era5, nome_arq)
        
        if os.path.exists(output_file):
            print(f"Arquivo {nome_arq} já existe. Verificando integridade...")
            if validate_netcdf(output_file):
                print(f"Arquivo {nome_arq} está válido. Pulando download.")
                continue
            else:
                print(f"Arquivo {nome_arq} corrompido. Redownload necessário.")
                os.remove(output_file)
        
        print(f"Baixando dados: {ano} - {mes}")
        
        try:
            retrieve_and_validate(ano, mes, output_file)

            if not process_downloaded_file(output_file):
                raise Exception("Erro ao processar arquivo baixado")
            
            # Validar arquivo após download
            if validate_netcdf(output_file):
                print(f"Download concluído e validado: {nome_arq}")
            else:
                print(f"Arquivo {nome_arq} corrompido após download")
                if os.path.exists(output_file):
                    os.remove(output_file)
                raise Exception("Arquivo corrompido")
                
        except Exception as e:
            error_msg = str(e).lower()
            print(f"Erro ao baixar para {ano}-{mes}: {e}")
            lst_errors.append((ano, mes, str(e)))
            
            # Remover arquivo corrompido se existir
            if os.path.exists(output_file):
                os.remove(output_file)
            
            # Tratamento de rate limiting
            if "429" in error_msg or "too many requests" in error_msg:
                print("Erro 429: Muitas requisições. Aguardando 1 hora...")
                time.sleep(3600)  # 1 hora
            elif "quota" in error_msg or "limit" in error_msg:
                print("Cota excedida. Aguardando 30 minutos...")
                time.sleep(1800)  # 30 minutos
            else:
                print("Aguardando 2 minutos antes da próxima tentativa...")
                time.sleep(120)  # 2 minutos
            continue

print("\nTodos os downloads concluídos!")
if lst_errors:
    print(f"Erros encontrados: {len(lst_errors)}")
    for erro in lst_errors:
        print(f"  {erro[0]}-{erro[1]}: {erro[2]}")
else:
    print("Nenhum erro encontrado!")

# Exemplo de como ler os arquivos após download
def ler_arquivo_era5(caminho_arquivo):
    """Função para ler arquivos ERA5 baixados"""
    try:
        ds = xr.open_dataset(caminho_arquivo)
        print(f"Arquivo carregado: {caminho_arquivo}")
        print(f"Variáveis disponíveis: {list(ds.variables)}")
        # Verifica qual dimensão temporal está disponível
        if 'time' in ds.dims:
            time_var = 'time'
        elif 'valid_time' in ds.dims:
            time_var = 'valid_time'
        else:
            time_var = None
            
        if time_var:
            print(f"Período: {ds[time_var].min().values} a {ds[time_var].max().values}")
        else:
            print("Dimensão temporal não encontrada")
        return ds
    except Exception as e:
        print(f"Erro ao ler arquivo {caminho_arquivo}: {e}")
        return None

# Exemplo de uso da função de leitura
print("\n--- Testando leitura dos arquivos baixados ---")
for arquivo in os.listdir(pasta_era5):
    if arquivo.endswith('.nc'):
        caminho_completo = os.path.join(pasta_era5, arquivo)
        ds = ler_arquivo_era5(caminho_completo)
        if ds is not None:
            ds.close()  # Importante: fechar o dataset após uso

netCDF4 disponível
Area de Download Calculada: [np.float64(-2.102599967999936), np.float64(-43.97358560699996), np.float64(-15.42714927299994), np.float64(-34.46749476699995)]


2025-07-24 19:58:53,375 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.


Download Iniciado...
Baixando dados: 2000 - 01


2025-07-24 19:58:54,455 INFO Request ID is b2956345-5403-4b42-87a2-90591443dcee
2025-07-24 19:58:54,717 INFO status has been updated to accepted
2025-07-24 20:05:20,829 INFO status has been updated to successful


Arquivo ZIP detectado: Dados\ERA5_Bruto\ERA5_NE-Brasil_2000-01.nc
Conteúdo do ZIP: ['data_stream-oper_stepType-instant.nc', 'data_stream-oper_stepType-accum.nc']
Fazendo merge de 2 arquivos NetCDF...
Carregando: data_stream-oper_stepType-instant.nc
Carregando: data_stream-oper_stepType-accum.nc
Combinando datasets...
Salvando arquivo combinado: Dados\ERA5_Bruto\ERA5_NE-Brasil_2000-01.nc
Merge concluído com sucesso
Arquivo válido. Dimensões: {'valid_time': 744, 'latitude': 54, 'longitude': 39}
Variáveis encontradas: ['u10', 'v10', 't2m', 'sp', 'tp']
Download concluído e validado: ERA5_NE-Brasil_2000-01.nc
Baixando dados: 2000 - 02


...

Arquivo ZIP detectado: Dados\ERA5_Bruto\ERA5_NE-Brasil_2025-05.nc
Conteúdo do ZIP: ['data_stream-oper_stepType-instant.nc', 'data_stream-oper_stepType-accum.nc']
Fazendo merge de 2 arquivos NetCDF...
Carregando: data_stream-oper_stepType-instant.nc
Carregando: data_stream-oper_stepType-accum.nc
Combinando datasets...
Merge falhou, tentando concatenar: conflicting values for variable 'expver' on objects to be combined. You can skip this check by specifying compat='override'.
Salvando arquivo combinado: Dados\ERA5_Bruto\ERA5_NE-Brasil_2025-05.nc
Merge concluído com sucesso
Arquivo válido. Dimensões: {'time': 2, 'valid_time': 744, 'latitude': 54, 'longitude': 39}
Variáveis encontradas: ['u10', 'v10', 't2m', 'sp', 'tp']
Download concluído e validado: ERA5_NE-Brasil_2025-05.nc
Baixando dados: 2025 - 06


2025-07-26 18:10:55,469 INFO Request ID is fcbc5b0a-102b-4689-8ccf-ce4a97187e0b
2025-07-26 18:10:55,742 INFO status has been updated to accepted
2025-07-26 18:11:10,980 INFO status has been updated to running
2025-07-26 18:19:20,563 INFO status has been updated to successful


Arquivo ZIP detectado: Dados\ERA5_Bruto\ERA5_NE-Brasil_2025-06.nc
Conteúdo do ZIP: ['data_stream-oper_stepType-instant.nc', 'data_stream-oper_stepType-accum.nc']
Fazendo merge de 2 arquivos NetCDF...
Carregando: data_stream-oper_stepType-instant.nc
Carregando: data_stream-oper_stepType-accum.nc
Combinando datasets...
Salvando arquivo combinado: Dados\ERA5_Bruto\ERA5_NE-Brasil_2025-06.nc
Merge concluído com sucesso
Arquivo válido. Dimensões: {'valid_time': 720, 'latitude': 54, 'longitude': 39}
Variáveis encontradas: ['u10', 'v10', 't2m', 'sp', 'tp']
Download concluído e validado: ERA5_NE-Brasil_2025-06.nc
Baixando dados: 2025 - 07


2025-07-26 18:19:44,735 INFO Request ID is b8391347-0dd1-41d6-b3a4-555e90ea5779
2025-07-26 18:19:45,081 INFO status has been updated to accepted
2025-07-26 18:19:59,795 INFO status has been updated to running
2025-07-26 18:26:08,403 INFO status has been updated to successful


Arquivo ZIP detectado: Dados\ERA5_Bruto\ERA5_NE-Brasil_2025-07.nc
Conteúdo do ZIP: ['data_stream-oper_stepType-instant.nc', 'data_stream-oper_stepType-accum.nc']
Fazendo merge de 2 arquivos NetCDF...
Carregando: data_stream-oper_stepType-instant.nc
Carregando: data_stream-oper_stepType-accum.nc
Combinando datasets...
Salvando arquivo combinado: Dados\ERA5_Bruto\ERA5_NE-Brasil_2025-07.nc
Merge concluído com sucesso
Arquivo válido. Dimensões: {'valid_time': 502, 'latitude': 54, 'longitude': 39}
Variáveis encontradas: ['u10', 'v10', 't2m', 'sp', 'tp']
Download concluído e validado: ERA5_NE-Brasil_2025-07.nc
Baixando dados: 2025 - 08
Erro ao baixar para 2025-08: 400 Client Error: Bad Request for url: https://cds.climate.copernicus.eu/api/retrieve/v1/processes/reanalysis-era5-single-levels/execution
invalid request
None of the data you have requested is available yet, please revise the period requested. The latest date available for this dataset is: 2025-07-21 21:00
Aguardando 2 minutos ant

In [ ]:
#Coleta de Renewable Ninjas

# Carregue seu dataset mestre final
master_path = os.path.join('Dados', 'Dataset_Locais.csv')
df_mestre = pd.read_csv(master_path)

# Agrupa por potência e altura e conta quantas turbinas existem em cada categoria
classes_de_turbinas = df_mestre.groupby(['potencia_turbina_mw', 'altura_torre_m']).size().reset_index(name='contagem')

MAPEAMENTO_TURBINAStemp =  classes_de_turbinas.to_dict(orient='records')
print(MAPEAMENTO_TURBINAStemp)
print(f"Total de classes de turbinas: {len(MAPEAMENTO_TURBINAStemp)}")
MAPEAMENTO_TURBINAS ={}

# Formato: (potencia_em_mw, altura_em_m): 'nome_do_modelo_no_site_do_ninja'
#for elem in MAPEAMENTO_TURBINAStemp:
    

TOKEN_LST = [
    '50fd6a8f5a429b2cd29222a19baade82f36f0b79',
    '8d3cc249efd7e25f5bf7d5a0bfb87be8b9848ba4',
    '00b576de51e9ccc5cf2806de6aad8244a0e970da',
    '56a5ac15cc619bf4775bcb289987a2c8d7fd2dc6',
    'ac70556c651b549c176d38b984812e205f939349',
    'd6a095da0d70715f76026f48a8789b7ebcf8fdda',
    '76b716ad2ffd8d8cd378c083990f4055fde0568a',
    '0bcca8208a622f8c589ac8b8581c9766edf9997a',
]

# --- Caminhos ---
master_path = os.path.join('Dados', 'Dataset_Locais.csv')
output_path = os.path.join('Dados', 'Ninjas_Bruto')
os.makedirs(output_path, exist_ok=True)


# ==============================================================================
# --- 2. FUNÇÃO DE DOWNLOAD E LÓGICA DE ROTAÇÃO DE TOKEN ---
# ==============================================================================

def fazer_requisicao_ninja(session, lat, lon, altura, modelo_turbina):
    """Faz uma única requisição à API do Renewables.ninja."""
    api_base = 'https://www.renewables.ninja/api/'
    url = api_base + 'data/wind'
    
    args = {
        'lat': lat, 'lon': lon,
        'date_from': '2000-01-01', 'date_to': '2024-12-31',
        'capacity': 1.0, 'height': altura,
        'turbine': modelo_turbina, 'format': 'csv',
        'header': True, 'local_time': False
    }
    
    r = session.get(url, params=args)
    # Lança um erro para status como 429, 500, etc., para que possamos tratá-los
    r.raise_for_status()
    return r.text

# --- 3. EXECUÇÃO ---

print("[INFO] Carregando dataset mestre para obter coordenadas representativas...")
df_mestre = pd.read_csv(master_path)
df_representantes = df_mestre.drop_duplicates(subset=['potencia_turbina_mw', 'altura_torre_m'])

# --- Inicialização da sessão e do token ---
token_pos = 0
session = requests.session()
session.headers = {'Authorization': 'Token ' + TOKEN_LST[token_pos]}
print(f"[INFO] Iniciando com o token #{token_pos + 1}")

print(f"[INFO] Iniciando download para {len(MAPEAMENTO_TURBINAS)} classes de turbinas...")

# Convertemos o dicionário para uma lista para poder re-tentar o item que falhou
classes_para_baixar = list(MAPEAMENTO_TURBINAS.items())
i = 0
while i < len(classes_para_baixar):
    (potencia, altura), modelo_ninja = classes_para_baixar[i]
    
    nome_arquivo = f"ninja_{potencia}MW_{altura}m.csv"
    caminho_saida = os.path.join(output_path, nome_arquivo)
    
    if os.path.exists(caminho_saida):
        print(f"[INFO] Arquivo para a classe {modelo_ninja} já existe. Pulando.")
        i += 1
        continue
    
    try:
        representante = df_representantes[
            (df_representantes['potencia_turbina_mw'] == potencia) &
            (df_representantes['altura_torre_m'] == altura)
        ].iloc[0]
        
        lat_req = representante['latitude_turbina']
        lon_req = representante['longitude_turbina']
        
        print(f"-> Requisitando dados para a classe: {modelo_ninja} a {altura}m...")
        dados_csv = fazer_requisicao_ninja(session, lat_req, lon_req, altura, modelo_ninja)
        
        with open(caminho_saida, 'w') as f:
            f.write(dados_csv)
            
        print(f"[SUCCESS] Dados para a classe '{modelo_ninja}' salvos.")
        i += 1 # Avança para a próxima classe APENAS se o download for bem-sucedido
        
    except requests.exceptions.HTTPError as e:
        # Verifica se o erro é de limite de requisições (429)
        if e.response.status_code == 429:
            print(f"[WARN] Limite de requisições atingido para o token #{token_pos + 1}.")
            token_pos += 1 # Tenta o próximo token
            
            if token_pos >= len(TOKEN_LST):
                print("[INFO] Todos os tokens foram usados. Pausando por 1 hora...")
                for _ in tqdm(range(3600), desc="Aguardando 1 hora"):
                    time.sleep(1)
                token_pos = 0 # Reinicia para o primeiro token
            
            # Atualiza o token na sessão e tenta a MESMA classe novamente
            print(f"[INFO] Trocando para o token #{token_pos + 1}.")
            session.headers.update({'Authorization': 'Token ' + TOKEN_LST[token_pos]})
            # Não incrementa 'i', para que o loop tente a mesma classe de novo
            
        else:
            # Se for outro erro HTTP (ex: 500 Internal Server Error), registra e pula
            print(f"[ERROR] Falha ao baixar dados para a classe {modelo_ninja}. Erro HTTP: {e}")
            i += 1 # Pula para a próxima classe
            
    except Exception as e:
        print(f"[ERROR] Falha inesperada para a classe {modelo_ninja}. Erro: {e}")
        i += 1 # Pula para a próxima classe

print("\n[INFO] Download por classes concluído!")

AttributeError: 'list' object has no attribute 'orderby'

In [ ]:
#Tratando dados do ERA5
import warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)



# Configurações

os.makedirs(serie_temporal, exist_ok=True)

# Constantes físicas
R_ESPECIFICO_AR_SECO = 287.058  # J/(kg·K)

print("Carregando coordenadas das turbinas...")
df_turbinas = dataset_locais

# Remove turbinas com coordenadas inválidas/NaN
df_turbinas = df_turbinas.dropna(subset=['latitude_turbina', 'longitude_turbina'])

# Para parques sem turbinas específicas, usa coordenadas do parque
mask_sem_turbina = df_turbinas['denominacao_aerogerador'].isna()
if mask_sem_turbina.any():
    print(f"Encontrados {mask_sem_turbina.sum()} parques sem turbinas específicas - usando coordenadas do parque")
    df_turbinas.loc[mask_sem_turbina, 'denominacao_aerogerador'] = df_turbinas.loc[mask_sem_turbina, 'CEG']

print(f"Total de pontos para extração: {len(df_turbinas)}")

# Lista arquivos ERA5 principais (não temporários)
pasta_era5 = [f for f in glob(os.path.join(pasta_era5, "*.nc")) if '_temp.nc' not in f]
print(f"Arquivos ERA5 encontrados: {len(pasta_era5)}")

def extrair_dados_pontos(ds, lat, lon):
    """Extrai dados para um ponto usando interpolação"""
    try:
        ponto = ds.interp(latitude=lat, longitude=lon, method='linear')
        ponto_df = ponto.to_dataframe().reset_index()
        return ponto_df
    except Exception as e:
        print(f"Erro ao extrair dados para ponto ({lat}, {lon}): {e}")
        return None

def processar_arquivo_era5(arquivo_nc, df_turbinas):
    """Processa arquivo ERA5 e seu par _temp.nc"""
    print(f"Processando: {os.path.basename(arquivo_nc)}")
    
    # --- INÍCIO DO TRECHO DE JUNÇÃO ---
    arquivo_principal = arquivo_nc 
    arquivo_temp = arquivo_principal.replace('.nc', '_temp.nc')
    
    # Verifica se ambos os arquivos existem
    if not os.path.exists(arquivo_temp):
        print(f"[WARN] Arquivo complementar {os.path.basename(arquivo_temp)} não encontrado. Seguindo sem concatenar.")
        arquivo_temp = None
    
    # Abre os dois datasets e os junta
    with xr.open_dataset(arquivo_principal) as ds_main:
        if (arquivo_temp is not None):
            with xr.open_dataset(arquivo_temp) as ds_temp:
                ds_completo = xr.merge([ds_main, ds_temp])
        else:
            ds_completo = ds_main
        
        # Padroniza dimensão temporal
        if 'valid_time' in ds_completo.dims:
            ds_completo = ds_completo.rename({'valid_time': 'time'})
        
        dados_extraidos = []
        
        for idx, turbina in tqdm(df_turbinas.iterrows(), total=len(df_turbinas),
                                desc="Extraindo dados das turbinas"):
            lat = turbina['latitude_turbina']
            lon = turbina['longitude_turbina']
            
            ponto_df = extrair_dados_pontos(ds_completo, lat, lon)
            
            if ponto_df is not None and not ponto_df.empty:

                ponto_df['CEG'] = turbina['CEG']
                ponto_df['denominacao_aerogerador'] = turbina['denominacao_aerogerador']
                
                
                
                # Engenharia de variáveis básica
                # 1. Calcular Velocidade e Direção do Vento
                
                ponto_df['velocidade_vento_100m'] = np.sqrt(ponto_df['u100']**2 + ponto_df['v100']**2)
                ponto_df['velocidade_vento_10m'] = np.sqrt(ponto_df['u10']**2 + ponto_df['v10']**2)
                # Direção em radianos, depois convertida para graus (0-360)
                
                ponto_df['direcao_vento_100m'] = np.mod(180 + np.rad2deg(np.arctan2(ponto_df['u100'], ponto_df['v100'])), 360)
                ponto_df['densidade_ar'] = ponto_df['sp'] / (R_ESPECIFICO_AR_SECO * ponto_df['t2m'])
                
                # 2. Calcular Wind Shear Exponent (alpha)
                # Evita divisão por zero se o vento a 10m for nulo
                # a = ln(v2/v1) / ln(h2/h1) -> ln(100/10) = 2.3025
                ponto_df['wind_shear'] = np.log(ponto_df['velocidade_vento_100m'] / ponto_df['velocidade_vento_10m'].replace(0, 0.01)) / 2.3025
                ponto_df['wind_shear'] = ponto_df['wind_shear'].fillna(0) # Preenche casos onde a velocidade é zero
                
                
                # 3. Calcular Intensidade da Turbulência (proxy)
                
                ponto_df['intensidade_turbulencia'] = ponto_df['zust'] / ponto_df['velocidade_vento_100m'].replace(0, 0.01)
                ponto_df['intensidade_turbulencia'] = ponto_df['intensidade_turbulencia'].fillna(0)

                ponto_df['potencia_vento_100m'] = 0.5 * ponto_df['densidade_ar'] * (ponto_df['velocidade_vento_100m']**3)
                
                
                # 4. Criar Variáveis Temporais
                ponto_df['time'] = pd.to_datetime(ponto_df['time'])
                ponto_df['hora_do_dia'] = ponto_df['time'].dt.hour
                ponto_df['dia_do_ano'] = ponto_df['time'].dt.dayofyear
                ponto_df['mes'] = ponto_df['time'].dt.month
                
                
                dados_extraidos.append(ponto_df)
        
        if dados_extraidos:
            df_completo = pd.concat(dados_extraidos, ignore_index=True)
            return df_completo
        else:
            return None
        #Fim do trecho de junção ---

# Processa todos os arquivos (usando append incremental)
print("Iniciando extração de dados...")

# Remove arquivo existente se houver
if os.path.exists(tratado_era5):
    os.remove(tratado_era5)
    print(f"Arquivo de saída antigo removido.")

for i, arquivo in enumerate(pasta_era5):
    df_arquivo = processar_arquivo_era5(arquivo, df_turbinas)
    if df_arquivo is not None and not df_arquivo.empty:
        if i == 0:
            # Primeiro arquivo: cria novo parquet
            df_arquivo.to_parquet(tratado_era5, index=False)
        else:
            # Arquivos seguintes: faz append usando fastparquet
            try:
                import fastparquet
                fastparquet.write(tratado_era5, df_arquivo, append=True)
            except ImportError:
                # Fallback: lê, concatena e salva (menos eficiente)
                df_existing = pd.read_parquet(tratado_era5)
                df_combined = pd.concat([df_existing, df_arquivo], ignore_index=True)
                df_combined.to_parquet(tratado_era5, index=False)

print("Extração concluída!")

# Verifica resultado final
if os.path.exists(tratado_era5):
    print("\nCarregando resultado final para estatísticas...")
    df_final = pd.read_parquet(tratado_era5)
    
    print(f"Dados extraídos:")
    print(f"- Total de registros: {len(df_final):,}")
    print(f"- Períodos temporais: {df_final['time'].nunique()}")
    print(f"- Turbinas/pontos processados: {df_final['denominacao_aerogerador'].nunique()}")
    print(f"- Variáveis disponíveis: {[col for col in df_final.columns if col not in ['time', 'CEG', 'denominacao_aerogerador', 'turbina']]}")
    
    # Cria versão pivotada para ML
    
    variaveis_para_pivotar = [
        'velocidade_vento_100m',
        'densidade_ar',
        'potencia_vento_100m',
        'direcao_vento_100m',
        'wind_shear',
        'intensidade_turbulencia',
        'tp' # Precipitação Total
    ]
    
    print("\nCriando versão pivotada...")
    
    print("\n--- Verificando o DataFrame antes do Pivot ---")
    print("Todas as colunas disponíveis no df_final:")
    print(df_final.columns.to_list())

    # Checa se as colunas necessárias para o cálculo existem
    colunas_necessarias = ['sp', 't2m']
    for col in colunas_necessarias:
        if col not in df_final.columns:
            print(f"\n[AVISO!] A coluna essencial '{col}' NÃO foi encontrada no DataFrame!")

    print("-------------------------------------------\n")
    
    df_pivotado = df_final.pivot_table(
        index='time',
        columns='denominacao_aerogerador',
        values=variaveis_para_pivotar,
        aggfunc='mean'  # Usa média se houver duplicatas
    ).reset_index()
    
    df_pivotado.columns = ['_'.join(col).strip() for col in df_pivotado.columns.values]
    df_pivotado.reset_index(inplace=True)
    
    output_pivot = os.path.join(serie_temporal, 'series_temporais_pivotado.parquet')
    df_pivotado.to_parquet(output_pivot, index=False)
    
    print(f"Dataset pivotado salvo: {df_pivotado.shape[0]} linhas x {df_pivotado.shape[1]-1} turbinas")
    print(f"Arquivos salvos em: {serie_temporal}")
    
else:
    print("Nenhum dado foi extraído. Verifique os arquivos ERA5.")

print("\nPasso 1 + Engenharia básica concluídos! - Faltando Engenharia Avancada e Tratamento")

Carregando coordenadas das turbinas...
Total de pontos para extração: 17171
Arquivos ERA5 encontrados: 307
Iniciando extração de dados...
Processando: ERA5_NE-Brasil_2000-01.nc
[WARN] Arquivo complementar ERA5_NE-Brasil_2000-01_temp.nc não encontrado. Seguindo sem concatenar.


Extraindo dados das turbinas: 100%|██████████| 17171/17171 [08:48<00:00, 32.49it/s]


Processando: ERA5_NE-Brasil_2000-02.nc
[WARN] Arquivo complementar ERA5_NE-Brasil_2000-02_temp.nc não encontrado. Seguindo sem concatenar.


Extraindo dados das turbinas: 100%|██████████| 17171/17171 [10:15<00:00, 27.89it/s]


Processando: ERA5_NE-Brasil_2000-03.nc
[WARN] Arquivo complementar ERA5_NE-Brasil_2000-03_temp.nc não encontrado. Seguindo sem concatenar.


...

Processando: ERA5_NE-Brasil_2011-08.nc
[WARN] Arquivo complementar ERA5_NE-Brasil_2011-08_temp.nc não encontrado. Seguindo sem concatenar.


Extraindo dados das turbinas: 100%|██████████| 17171/17171 [17:37<00:00, 16.23it/s] 


Processando: ERA5_NE-Brasil_2011-09.nc
[WARN] Arquivo complementar ERA5_NE-Brasil_2011-09_temp.nc não encontrado. Seguindo sem concatenar.


Extraindo dados das turbinas: 100%|██████████| 17171/17171 [14:19<00:00, 19.98it/s] 


OSError: [Errno 28] No space left on device

In [ ]:
#Junção dos DataSets -> Criação do DF MASTER


destino_drive = r'G://content//drive//MyDrive//PUC//IC//ColetaEra5//Dados'
TEMP_DIR        = r"C:"  #Editar depois


CUT_IN, RATED, CUT_OUT = 3.0, 12.0, 25.0
FATOR_PERDAS = 0.85
MEM_LIMIT = "4GB"   # ajuste conforme a sua RAM


Path(destino_drive).mkdir(parents=True, exist_ok=True)
Path(TEMP_DIR).mkdir(parents=True, exist_ok=True)

LOG_PATH = Path(destino_drive) / "processo_mes_a_mes.log"
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    handlers=[logging.FileHandler(LOG_PATH, encoding="utf-8"), logging.StreamHandler()]
)
log = logging.getLogger("mes-a-mes")

def limites_do_mes(ano:int, mes:int):
    """Retorna strings [inicio, fim) para o mês."""
    inicio = f"{ano}-{mes:02d}-01 00:00:00"
    if mes == 12:
        fim = f"{ano+1}-01-01 00:00:00"
    else:
        fim = f"{ano}-{mes+1:02d}-01 00:00:00"
    return inicio, fim

def mes_ja_processado(ano:int, mes:int) -> bool:
    """Se já existir 'ano=YYYY/mes=M' com pelo menos 1 parquet, pula (idempotência)."""
    base = Path(destino_drive) / f"ano={ano}" / f"mes={mes}"
    if not base.exists():
        return False
    return any(p.suffix.lower() == ".parquet" for p in base.rglob("*.parquet"))


def processar_mes(ano:int, mes:int):
    inicio = f"{ano}-{mes:02d}-01 00:00:00"
    if mes == 12:
        prox_ano, prox_mes = ano +1, 1
    else:
        prox_ano, prox_mes = ano, mes +1
    fim = f"{prox_ano}-{prox_mes:02d}-01 00:00:00"

    con = duckdb.connect(database=":memory:")
    con.execute(f"PRAGMA memory_limit= '{MEM_LIMIT}';")
    con.execute("PRAGMA threads=8;")
    con.execute(f"PRAGMA temp_directory='{TEMP_DIR}';")
    con.execute("PRAGMA enable_progress_bar;")

    con.execute(f"""
        CREATE TEMP TABLE s AS
        SELECT
            CEG,
            denominacao_aerogerador,
            altura_torre_m,
            diametro_rotor_m,
            potencia_turbina_mw,
            nome_parque,
            status_operacao,
            ano_inicio_operacao
        FROM read_csv_auto('{dataset_locais}', SAMPLE_SIZE=-1);
    """)

     # Seleciona SOMENTE colunas necessárias do Parquet e filtra o mês
    # Calcula a potência como no seu código (modelo simplificado com limite na nominal)
    # Particiona por ano/mes/dia para gerar arquivos menores.
    con.execute(f"""
        COPY (
            WITH src AS (
                SELECT
                    time,
                    CEG,
                    denominacao_aerogerador,
                    velocidade_vento_100m,
                    densidade_ar,
                    TRY_CAST(time AS TIMESTAMP) AS ts
                FROM parquet_scan('{tratado_era5}')
                WHERE TRY_CAST(time AS TIMESTAMP) >= TIMESTAMP '{inicio}'
                  AND TRY_CAST(time AS TIMESTAMP) <  TIMESTAMP '{fim}'
            )
            SELECT
                t.time,
                t.CEG,
                t.denominacao_aerogerador,
                t.velocidade_vento_100m,
                s.altura_torre_m,
                s.diametro_rotor_m,
                s.potencia_turbina_mw,
                s.nome_parque,
                s.status_operacao,
                s.ano_inicio_operacao,
                EXTRACT(YEAR  FROM t.ts) AS ano,
                EXTRACT(MONTH FROM t.ts) AS mes,
                EXTRACT(DAY   FROM t.ts) AS dia,
                (
                    CASE
                        WHEN t.velocidade_vento_100m >= {RATED} AND t.velocidade_vento_100m < {CUT_OUT}
                            THEN s.potencia_turbina_mw
                        WHEN t.velocidade_vento_100m >= {CUT_IN} AND t.velocidade_vento_100m < {RATED}
                            THEN LEAST(
                                (0.5 * t.densidade_ar * (t.velocidade_vento_100m * t.velocidade_vento_100m * t.velocidade_vento_100m) * (pi() * (s.diametro_rotor_m/2) * (s.diametro_rotor_m/2))) / 1e6,
                        s.potencia_turbina_mw
                            )
                        ELSE 0.0
                    END
                ) * {FATOR_PERDAS} AS potencia_gerada_estimada_mw
            FROM src t
            LEFT JOIN s USING (CEG, denominacao_aerogerador)
        )
        TO '{destino_drive}'
        (FORMAT PARQUET,
         COMPRESSION ZSTD,
         PARTITION_BY (ano, mes, dia),
         ROW_GROUP_SIZE 250000,
         OVERWRITE_OR_IGNORE 1);
    """)

if __name__ == "__main__":
    pares = []
    for ano in range(2000, 2026):
        fim_mes = 7 if ano ==2025 else 12
        for mes in range(1,fim_mes + 1):
            pares.append((ano, mes))
    total = len(pares)
    falhas = []

    with tqdm( total=total, desc='Processando meses', unit="mês") as barra:
        for (ano, mes) in pares:
            try:
                if mes_ja_processado(ano, mes):
                    log.info(f"Pulanod {ano}-{mes:02d} (já processado)")
                else:
                    #log.info(f"Iniciando {ano}-{mes:02d}")
                    processar_mes(ano,mes)
                    #log.info(f"Concluido {ano}-{mes:02d}")
            except Exception as e:
                falhas.append((ano, mes, str(e)))
                log.exception(f"Erro em {ano}-{mes:02d}: {e}")
            finally:
                barra.update(1)
    if falhas:
        log.warning("Meses com falha:")
        for ano, mes, err in falhas:
            log.warning(f"  {ano}-{mes:02d}: {err}")
        print(f"Concluido com falhas em {len(falhas)}. Consulte o log: {LOG_PATH}")
    else:
        print("Concluido sem falhas!!!")

Pipeline:   0%|          | 0/5 [00:00<?, ?etapa/s]

• Carregar séries temporais

Pipeline:  20%|██        | 1/5 [00:16<01:07, 16.91s/etapa]

• Carregar dados estáticos

Pipeline:  40%|████      | 2/5 [00:17<00:21,  7.08s/etapa]

...

Pipeline: 100%|██████████| 5/5 [01:02<00:00, 12.50s/etapa]


  ✓ PROCESSO CONCLUÍDO  

 Arquivo salvo                              ┃     Linhas ┃ Colunas 
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━
 Dados\Dataset_Final_Para_Modelagem.parquet │ 12,410,640 │      33

Prévia das 5 primeiras linhas:

time denominacao_aerogerador  velocidade_vento_100m  potencia_turbina_mw  
potencia_gerada_estimada_mw
2024-06-01 00:00:00                 INH-045               7.405772                 1.85                     
1.023743
2024-06-01 01:00:00                 INH-045               7.146365                 1.85                     
0.922049
2024-06-01 02:00:00                 INH-045               6.972974                 1.85                     
0.857884
2024-06-01 03:00:00                 INH-045               6.876829                 1.85                     
0.824687
2024-06-01 04:00:00                 INH-045               6.824042                 1.85                     
0.806535